In [1]:
import qmcpy as qp
import numpy as np
import scipy.stats
import math
#import pandas as pd
import time
#from matplotlib import pyplot
#pyplot.style.use('../qmcpy/qmcpy.mplstyle')
#colors = pyplot.rcParams['axes.prop_cycle'].by_key()['color']
print('ok')

ok


We generated our points in $d$ dimensions and generated $n$ samples such that $\boldsymbol{x}_i \in [0,1]^d$ for $i = 1, 2, 3, ..., n$. Let matrix $\boldsymbol{X}$ be a $n \times d$ matrix such that
$$
\boldsymbol{X} = 
\begin{bmatrix}
\boldsymbol{x}_1\\
\boldsymbol{x}_2\\
\boldsymbol{x}_3\\
...\\
\boldsymbol{x}_n\\
\end{bmatrix}
$$
which can be rewritten as
$$
\boldsymbol{X} = 
\begin{bmatrix}
\boldsymbol{x}_{1_1} & \boldsymbol{x}_{1_2} & \boldsymbol{x}_{1_3} & ... & \boldsymbol{x}_{1_d}\\
\boldsymbol{x}_{2_1} & \boldsymbol{x}_{2_2} & \boldsymbol{x}_{2_3} & ... & \boldsymbol{x}_{2_d}\\
\boldsymbol{x}_{3_1} & \boldsymbol{x}_{3_2} & \boldsymbol{x}_{3_3} & ... & \boldsymbol{x}_{3_d}\\
... & ... & ... & ... & ...\\
\boldsymbol{x}_{n_1} & \boldsymbol{x}_{n_2} & \boldsymbol{x}_{n_3} & ... & \boldsymbol{x}_{n_d}\\
\end{bmatrix}
$$

We want to find our discrepancy. There is a formula we can use which is
$$
[D(\boldsymbol{X}) ]^2= \underbrace{\int_{[0,1]^d} \int_{[0,1]^d} K(x,y)~dx~dy}_{A} - \frac{2}{n} \sum_{i=1}^n \underbrace{\int_{[0,1]^d} K(x, \boldsymbol{x}_i) ~dx}_{B(\boldsymbol{x}_i)} + \frac{1}{n^2} \sum_{i,j = 1}^n K(\boldsymbol{x}_i, \boldsymbol{x}_j)
$$
Let $A$ be the double integral, $B$ the single integral, and $C$ be the Kernel. Of course, this depends on which discrepancy method you want to use. At which there are several. We will start off with L2star.

In [6]:
A = qp.IIDStdUniform(dimension=2)
X = A.gen_samples(n=2**12)
print(qp.discrepancy(X, 'l2star'))

0.010808211428705361


So, you would input your array of samples and method respectively. As for other methods for calculating discrepancy in our library, we got:

 symmetric
 centered
 wrap around
 mixture

However, do you want to use a different method that isn't in our library. There is an easy fix to that. Unanchored Discrepancy is not in our code. We define our double_integral "$A$", single_integral "$B$", and kernel "$C$". We code,

In [9]:
A = lambda w: (13/12)**len(w)
B = lambda x, w: (1+ ((x*(1-x))/2)).prod(axis=1)
C = lambda x, y, w: (1 + ((np.minimum(x, y) - (x*y)))).prod(axis=2)
method = [A, B, C]
print(qp.discrepancy(X, method))

0.009415807604203021


This is just unweighted unanchored discrepancy. Possibly you may want to use weights in determining a weighted discrepancy for these samples. Of course, the weighted discrepancy for unanchored discrepancy is:

In [10]:
A = lambda w: (1 + (w/12)).prod()
B = lambda x, w: (1+ (w*(x*(1-x))/2)).prod(axis=1)
C = lambda x, y, w: (1 + (w*(np.minimum(x, y) - (x*y)))).prod(axis=2)

Now, you might want to weigh the discrepancy with $weight = [1, 1/2]$ and to show that these values are different let $weight = [1, 1/3]$. We calculate,

In [11]:
method = [A, B, C]
weight1 = np.array([1, .5])
weight2 = np.array([1, 1/3])
print(qp.discrepancy(X, method, weight = weight1))
print(qp.discrepancy(X, method, weight = weight2))

0.008887238713788308
0.008703918944714962


This works the same way if you allow method to be either L2star, centered, mixture, warp around, or symmetric. We have a parameter called limiter. It's purpose is to limit the computer to do however many operations such that it doesn't get overwhelmed and not running out of memory space. So instead of doing all the computations all at once, we only going piece by piece. Depending on your computer you should change your limiter. This may also effect how much time it takes for the computer to compute them. Of course, we got Time as a parameter to show that it does effect the runtime. We will change limiter in iterations of 5. With Time = True, the 2nd paramter would give us our runtime.

In [17]:
print(qp.discrepancy(X, method, limiter = 2**35, Time = True))
print(qp.discrepancy(X, method, limiter = 2**30, Time = True))
print(qp.discrepancy(X, method, limiter = 2**25, Time = True))
print(qp.discrepancy(X, method, limiter = 2**20, Time = True))
print(qp.discrepancy(X, method, limiter = 2**15, Time = True))
print(qp.discrepancy(X, method, limiter = 2**10, Time = True))
print(qp.discrepancy(X, method, limiter = 2**5, Time = True))

[0.009415807604203021, 1.4748549461364746]
[0.009415807604203021, 1.3715436458587646]
[0.009415807604203021, 1.4231503009796143]
[0.009415807604108694, 0.8651859760284424]
[0.009415807604014365, 0.4437375068664551]
[0.009415807604203021, 0.9483113288879395]
[0.009415807603377648, 15.876707792282104]


In [5]:
x = [0, 1, 2, 3]
y = [0, 1, 4, 9]
x_1 = [0, 1, 2, 3]
y_1 = [0, 1, 8, 27]

# Create the plot
plt.plot(x, y)
plt.plot(x_1, y_1)
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.title("2D Graph Example")
plt.grid(True)  # Add grid lines
plt.show()

NameError: name 'plt' is not defined

In [7]:
w = np.array([1, 2, 3])
print(w)
print(len(w))

[1 2 3]
3
